In [1]:
import datetime
import time
import json
import os
import string
import requests
import sys
import traceback

import azure.cosmos.cosmos_client as cosmos_client
import pandas as pd

from arcgis import GIS
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from helpers import keys

In [2]:
gis = GIS('http://esrifederal.maps.arcgis.com', keys.user_name, keys.password)
print("Successfully connected to ArcGIS Online...")

Successfully connected to ArcGIS Online...


In [3]:
embassy_locations = gis.content.get('e17b84b94cb24fea83bf402b756e4f95')
print("Connected to Embassies data layer")

Connected to Embassies data layer


In [5]:
flayer = embassy_locations.layers[0]
df = flayer.query(where='1=1').sdf

In [6]:
config = {
    'ENDPOINT': keys.cosmos_uri,
    'PRIMARYKEY': keys.cosmos_primary_key,
    'DATABASE': 'testdb',
    'CONTAINER': 'sample'
}

In [7]:
print("Connecting to Cosmos DB SQL API...")
client = cosmos_client.CosmosClient(url_connection=config['ENDPOINT'], auth={
                                    'masterKey': config['PRIMARYKEY']})

Connecting to Cosmos DB SQL API...


In [8]:
db_id = config['DATABASE']
db_query = "select * from r where r.id = '{0}'".format(db_id)
db = list(client.QueryDatabases(db_query))[0]
db_link = db['_self']
print("Connected to {0}...".format(db_id))

coll_id = config['CONTAINER']
coll_query = "select * from r where r.id = '{0}'".format(coll_id)
coll = list(client.QueryContainers(db_link, coll_query))[0]
coll_link = coll['_self']
print("Connected to {0}...".format(coll_id))

Connected to testdb...
Connected to sample...


In [9]:
docs = client.ReadItems(coll_link)
for item in docs:
    city = item['name']
    query_df = df.loc[df['ConSvcProv'] == city]
    lat = query_df['Lat'].item()
    long = query_df['Long_'].item()
    print("Location of {0}: {1}, {2}".format(city, str(long), str(lat)))
    
    new_cosmos_item = client.CreateItem(coll_link,{
                    'id':item['id'] + "1",
                    'name': item['name'],
                    'lat': lat,
                    'long':long
                        })
    print("Successfully added new item...")

Location of London: -0.0125737, 51.508529
Successfully added new item...
Location of Athens: 23.742804, 38.033329
Successfully added new item...
